# Land Use Change Analysis
## Archetype 2: The Ecologist

This notebook demonstrates how to analyze land use change using satellite data, a key skill for ecologists studying the drivers of zoonotic spillover. Students will analyze deforestation rates over time and correlate these changes with wildlife habitats to identify potential spillover hotspots.

### Learning Objectives:
- Understand the relationship between land use change and zoonotic disease risk
- Learn to work with geospatial data and satellite imagery
- Analyze deforestation patterns and their ecological impacts
- Identify potential zoonotic spillover risk areas
- Apply the "One Health" framework connecting human, animal, and environmental health

### Key Concepts:
- **Zoonotic spillover**: Transmission of pathogens from animals to humans
- **Habitat fragmentation**: Breaking up of continuous habitats into smaller patches
- **Edge effects**: Changes in environmental conditions at habitat boundaries
- **Biodiversity dilution effect**: How species diversity can reduce disease transmission
- **Land use/land cover (LULC)**: Classification of Earth's surface into different categories

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
from scipy import ndimage
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

## Part 1: Understanding Land Use Change and Disease Risk

Land use change is one of the primary drivers of emerging infectious diseases. When humans alter natural landscapes, they create new interfaces between wildlife and human populations, increasing opportunities for pathogen spillover.

### Key Mechanisms:
1. **Habitat destruction** forces wildlife into closer contact with humans
2. **Edge effects** create new ecological niches that may favor certain species
3. **Biodiversity loss** can reduce the "dilution effect" that protects against disease
4. **Stress on wildlife** may make them more susceptible to infection and shedding

In [ ]:
# Generate synthetic satellite data for a tropical region
# This simulates what you might get from Landsat or other satellite imagery

def generate_landscape_data(size=100, years=10):
    """
    Generate synthetic land use data over time
    0 = Water, 1 = Urban, 2 = Agriculture, 3 = Forest, 4 = Grassland
    """
    np.random.seed(42)  # For reproducibility
    
    # Initial landscape - mostly forest with some water and grassland
    landscape_2010 = np.random.choice([0, 3, 4], size=(size, size), p=[0.1, 0.7, 0.2])
    
    landscapes = {'2010': landscape_2010.copy()}
    current_landscape = landscape_2010.copy()
    
    # Simulate deforestation and agricultural expansion over time
    for year in range(2011, 2020):
        # Convert some forest to agriculture (deforestation)
        forest_pixels = np.where(current_landscape == 3)
        
        if len(forest_pixels[0]) > 0:
            # Select random forest pixels to convert
            n_convert = min(int(len(forest_pixels[0]) * 0.03), len(forest_pixels[0]))  # 3% per year
            indices = np.random.choice(len(forest_pixels[0]), n_convert, replace=False)
            
            for i in indices:
                row, col = forest_pixels[0][i], forest_pixels[1][i]
                # 70% agriculture, 30% urban
                current_landscape[row, col] = np.random.choice([2, 1], p=[0.7, 0.3])
        
        landscapes[str(year)] = current_landscape.copy()
    
    return landscapes

# Generate the landscape data
landscapes = generate_landscape_data()

# Define land use categories and colors
land_use_categories = {
    0: 'Water',
    1: 'Urban',
    2: 'Agriculture', 
    3: 'Forest',
    4: 'Grassland'
}

colors = ['blue', 'gray', 'yellow', 'green', 'lightgreen']
cmap = plt.matplotlib.colors.ListedColormap(colors)

print("✅ Generated synthetic landscape data for 2010-2019")
print(f"📍 Study area: {landscapes['2010'].shape[0]}x{landscapes['2010'].shape[1]} pixels")
print(f"🗓️ Time series: {len(landscapes)} years")

## Part 2: Visualizing Land Use Change Over Time

In [ ]:
# Create a time series visualization of land use change
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

years = list(landscapes.keys())
for i, year in enumerate(years):
    im = axes[i].imshow(landscapes[year], cmap=cmap, vmin=0, vmax=4)
    axes[i].set_title(f'Year {year}', fontsize=12, fontweight='bold')
    axes[i].set_xticks([])
    axes[i].set_yticks([])

# Add colorbar
cbar = plt.colorbar(im, ax=axes, orientation='horizontal', pad=0.1, shrink=0.8)
cbar.set_ticks(range(5))
cbar.set_ticklabels(list(land_use_categories.values()))

plt.suptitle('Land Use Change Over Time (2010-2019)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("🌍 Satellite imagery simulation showing deforestation and agricultural expansion")
print("🟢 Green = Forest, 🟡 Yellow = Agriculture, ⬜ Gray = Urban, 🔵 Blue = Water")

## Part 3: Quantitative Analysis of Land Use Change

In [ ]:
# Calculate land use statistics over time
def calculate_land_use_stats(landscapes):
    """
    Calculate the percentage of each land use type over time
    """
    stats = []
    
    for year, landscape in landscapes.items():
        total_pixels = landscape.size
        year_stats = {'Year': int(year)}
        
        for code, category in land_use_categories.items():
            count = np.sum(landscape == code)
            percentage = (count / total_pixels) * 100
            year_stats[category] = percentage
        
        stats.append(year_stats)
    
    return pd.DataFrame(stats)

# Calculate statistics
land_use_stats = calculate_land_use_stats(landscapes)
print("📊 Land Use Statistics:")
print(land_use_stats.round(2))

In [ ]:
# Create time series plots of land use change
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: All land use types over time
for category in ['Forest', 'Agriculture', 'Urban', 'Grassland', 'Water']:
    ax1.plot(land_use_stats['Year'], land_use_stats[category], 
             marker='o', linewidth=2.5, markersize=6, label=category)

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Percentage of Landscape (%)', fontsize=12)
ax1.set_title('Land Use Change Over Time', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Focus on forest loss and agricultural expansion
ax2.plot(land_use_stats['Year'], land_use_stats['Forest'], 
         'g-', marker='o', linewidth=3, markersize=8, label='Forest Cover')
ax2.plot(land_use_stats['Year'], land_use_stats['Agriculture'], 
         'y-', marker='s', linewidth=3, markersize=8, label='Agricultural Land')

ax2.set_xlabel('Year', fontsize=12)
ax2.set_ylabel('Percentage of Landscape (%)', fontsize=12)
ax2.set_title('Deforestation and Agricultural Expansion', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add annotation for total forest loss
forest_2010 = land_use_stats[land_use_stats['Year'] == 2010]['Forest'].iloc[0]
forest_2019 = land_use_stats[land_use_stats['Year'] == 2019]['Forest'].iloc[0]
forest_loss = forest_2010 - forest_2019

ax2.annotate(f'Total Forest Loss:\n{forest_loss:.1f} percentage points', 
             xy=(2015, forest_2019 + 5), fontsize=11,
             bbox=dict(boxstyle="round,pad=0.3", facecolor='lightcoral', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"\n🌳 Key Findings:")
print(f"Forest cover decreased from {forest_2010:.1f}% to {forest_2019:.1f}% ({forest_loss:.1f} percentage point loss)")
agr_2010 = land_use_stats[land_use_stats['Year'] == 2010]['Agriculture'].iloc[0]
agr_2019 = land_use_stats[land_use_stats['Year'] == 2019]['Agriculture'].iloc[0]
print(f"Agricultural land increased from {agr_2010:.1f}% to {agr_2019:.1f}% (+{agr_2019-agr_2010:.1f} percentage points)")

## Part 4: Habitat Fragmentation Analysis

Habitat fragmentation increases edge effects and can create conditions favorable for zoonotic spillover. We'll analyze how forest fragmentation has changed over time.

In [ ]:
def calculate_fragmentation_metrics(landscape, target_class=3):
    """
    Calculate habitat fragmentation metrics
    target_class: land use class to analyze (3 = Forest)
    """
    # Create binary mask for target habitat
    habitat_mask = (landscape == target_class).astype(int)
    
    # Label connected components (forest patches)
    labeled_patches, num_patches = ndimage.label(habitat_mask)
    
    # Calculate patch sizes
    patch_sizes = []
    for i in range(1, num_patches + 1):
        patch_size = np.sum(labeled_patches == i)
        patch_sizes.append(patch_size)
    
    # Calculate edge pixels (forest pixels adjacent to non-forest)
    # Use erosion to find interior pixels, then subtract to get edge pixels
    eroded = ndimage.binary_erosion(habitat_mask)
    edge_pixels = habitat_mask - eroded
    edge_length = np.sum(edge_pixels)
    
    metrics = {
        'num_patches': num_patches,
        'total_habitat': np.sum(habitat_mask),
        'mean_patch_size': np.mean(patch_sizes) if patch_sizes else 0,
        'largest_patch': np.max(patch_sizes) if patch_sizes else 0,
        'edge_length': edge_length,
        'edge_density': edge_length / np.sum(habitat_mask) if np.sum(habitat_mask) > 0 else 0
    }
    
    return metrics, labeled_patches

# Calculate fragmentation metrics for all years
fragmentation_data = []
patch_maps = {}

for year, landscape in landscapes.items():
    metrics, patches = calculate_fragmentation_metrics(landscape)
    metrics['Year'] = int(year)
    fragmentation_data.append(metrics)
    patch_maps[year] = patches

fragmentation_df = pd.DataFrame(fragmentation_data)
print("🧩 Forest Fragmentation Analysis:")
print(fragmentation__df.round(2))

In [ ]:
# Visualize fragmentation changes
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Number of forest patches over time
axes[0,0].plot(fragmentation_df['Year'], fragmentation_df['num_patches'], 
               'ro-', linewidth=3, markersize=8)
axes[0,0].set_xlabel('Year')
axes[0,0].set_ylabel('Number of Forest Patches')
axes[0,0].set_title('Forest Fragmentation Over Time')
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Mean patch size over time
axes[0,1].plot(fragmentation_df['Year'], fragmentation_df['mean_patch_size'], 
               'go-', linewidth=3, markersize=8)
axes[0,1].set_xlabel('Year')
axes[0,1].set_ylabel('Mean Patch Size (pixels)')
axes[0,1].set_title('Average Forest Patch Size')
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Edge density over time
axes[1,0].plot(fragmentation_df['Year'], fragmentation_df['edge_density'], 
               'bo-', linewidth=3, markersize=8)
axes[1,0].set_xlabel('Year')
axes[1,0].set_ylabel('Edge Density (edge pixels / habitat pixels)')
axes[1,0].set_title('Forest Edge Density')
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Comparison of 2010 vs 2019 patch structure
# Show 2010 patches
patches_2010 = patch_maps['2010']
patches_2019 = patch_maps['2019']

# Create a visualization showing patch boundaries
forest_2019 = (landscapes['2019'] == 3).astype(float)
forest_2010 = (landscapes['2010'] == 3).astype(float)

# Overlay 2010 forest boundaries on 2019 landscape
overlay = forest_2019.copy()
overlay[forest_2010 == 1] = 0.5  # Show original forest in different shade

axes[1,1].imshow(overlay, cmap='RdYlGn', alpha=0.8)
axes[1,1].set_title('Forest Loss Pattern\n(Dark Green=2019 Forest, Light=2010 Only)')
axes[1,1].set_xticks([])
axes[1,1].set_yticks([])

plt.tight_layout()
plt.show()

# Print key fragmentation insights
print(f"\n🔍 Fragmentation Analysis Results:")
patches_2010 = fragmentation_df[fragmentation_df['Year'] == 2010]['num_patches'].iloc[0]
patches_2019 = fragmentation_df[fragmentation_df['Year'] == 2019]['num_patches'].iloc[0]
print(f"Forest patches increased from {patches_2010} to {patches_2019} (more fragmentation)")

edge_2010 = fragmentation_df[fragmentation_df['Year'] == 2010]['edge_density'].iloc[0]
edge_2019 = fragmentation_df[fragmentation_df['Year'] == 2019]['edge_density'].iloc[0]
print(f"Edge density increased from {edge_2010:.3f} to {edge_2019:.3f} (+{((edge_2019/edge_2010)-1)*100:.1f}%)")
print(f"This indicates more forest-human interface area where spillover could occur")

## Part 5: Wildlife Habitat Suitability and Spillover Risk Assessment

Now we'll simulate how different wildlife species respond to land use change and identify potential spillover hotspots.

In [ ]:
# Define habitat preferences for different wildlife species
species_profiles = {
    'Forest Specialist': {3: 1.0, 4: 0.2, 2: 0.0, 1: 0.0, 0: 0.0}, # Loves forest, avoids humans
    'Edge Generalist': {3: 0.6, 4: 0.8, 2: 0.7, 1: 0.4, 0: 0.0}, # Thrives in mixed landscapes
    'Agricultural Specialist': {3: 0.1, 4: 0.5, 2: 1.0, 1: 0.1, 0: 0.0} # Adapted to farms
}

def calculate_habitat_suitability(landscape, profile):
    """Calculate habitat suitability map for a species"""
    suitability_map = np.zeros_like(landscape, dtype=float)
    for land_use, score in profile.items():
        suitability_map[landscape == land_use] = score
    return suitability_map

# Calculate spillover risk (high where edge generalists meet human activity)
def calculate_spillover_risk(landscape, edge_generalist_profile):
    """
    Calculate spillover risk based on habitat suitability and proximity to humans
    """
    # Get habitat suitability for the high-risk species (Edge Generalist)
    generalist_suitability = calculate_habitat_suitability(landscape, edge_generalist_profile)
    
    # Create a mask for human activity (Urban and Agriculture)
    human_activity_mask = np.isin(landscape, [1, 2])
    
    # Calculate distance from each pixel to the nearest human activity pixel
    distance_to_human = ndimage.distance_transform_edt(~human_activity_mask)
    
    # Risk is high where generalist suitability is high and distance to humans is low
    # We invert the distance so that small distance = high score
    proximity_to_human_score = np.exp(-distance_to_human / 5) # Exponential decay
    
    # Combine the two factors
    spillover_risk_map = generalist_suitability * proximity_to_human_score
    
    # Normalize risk score to be between 0 and 1
    if spillover_risk_map.max() > 0:
        spillover_risk_map /= spillover_risk_map.max()
        
    return spillover_risk_map

# Analyze the landscape for the final year (2019)
landscape_2019 = landscapes['2019']
spillover_risk_map_2019 = calculate_spillover_risk(landscape_2019, species_profiles['Edge Generalist'])

# Visualize the results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Original land use map for 2019
axes[0,0].imshow(landscape_2019, cmap=cmap, vmin=0, vmax=4)
axes[0,0].set_title('Land Use Map (2019)', fontsize=14, fontweight='bold')
axes[0,0].set_xticks([])
axes[0,0].set_yticks([])

# Plots 2-4: Habitat suitability for each species
suitability_maps = {}
for i, (species, profile) in enumerate(species_profiles.items()):
    ax = axes.flatten()[i+1]
    suitability_map = calculate_habitat_suitability(landscape_2019, profile)
    suitability_maps[species] = suitability_map
    im = ax.imshow(suitability_map, cmap='viridis', vmin=0, vmax=1)
    ax.set_title(f'{species} Habitat Suitability', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.tight_layout()
plt.show()

# Visualize the final spillover risk map
plt.figure(figsize=(10, 10))
plt.imshow(spillover_risk_map_2019, cmap='Reds', interpolation='bilinear')
plt.title('Zoonotic Spillover Risk Hotspots (2019)', fontsize=16, fontweight='bold')
plt.colorbar(label='Normalized Risk Score')
plt.xticks([])
plt.yticks([])
plt.show()

# Print summary
print("\n🦇 Spillover Risk Analysis Complete:")
print("The spillover risk map highlights areas where 'Edge Generalist' species, potential pathogen carriers, are most likely to come into contact with human activities like farming and urban settlements.")
print("These hotspots are concentrated at the fragmented boundaries between forests and agricultural land.")

## Part 6: Conclusion and Mitigation Strategies

This analysis demonstrates how deforestation and habitat fragmentation can be quantified to assess zoonotic spillover risk. The identified hotspots at the human-wildlife interface are critical areas for public health surveillance and ecological conservation.

### Potential Mitigation Strategies:
- **Create Buffer Zones**: Establish protected buffer zones around remaining forest patches to limit direct contact between wildlife and human settlements.
- **Promote Agroforestry**: Encourage farming practices that integrate trees and shrubs, creating softer edges between forests and agriculture.
- **Wildlife Corridors**: Connect fragmented forest patches with wildlife corridors to allow species to move without entering human-dominated areas.
- **One Health Surveillance**: Implement integrated surveillance systems that monitor pathogens in wildlife, domestic animals, and humans in high-risk zones.